In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.image as mpimg
sns.set(style = "ticks")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("final_0902.csv")

### Importing the dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2433 entries, 0 to 2432
Data columns (total 59 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Control                                   2433 non-null   object 
 1   State                                     2433 non-null   object 
 2   Plan                                      2433 non-null   object 
 3   2018 Fall Enrollment                      2433 non-null   float64
 4   Known_Cases_in_County_per_100k_Residents  2433 non-null   int64  
 5   lat                                       2433 non-null   float64
 6   lng                                       2433 non-null   float64
 7   institution                               2433 non-null   object 
 8   Address                                   2433 non-null   object 
 9   Campus_Size                               2433 non-null   object 
 10  Housing                             

### Handling Object Columns

Looks like we have a few columns that are supposed to be Numeric but coded as Object. Let's see what they are 

In [4]:
df.select_dtypes(include = "object").columns

Index(['Control', 'State', 'Plan', 'institution', 'Address', 'Campus_Size',
       'Housing', 'Total_Federal_Student_Loans',
       'AVG_Dederal_Student_Loans_Amount', 'college_year', 'city', 'county',
       'state', 'Total_Cases', 'Perecent_of_State_Cases', 'Cases_per_100000',
       'Total_Deaths'],
      dtype='object')

It seems that "Total_Federal_Student_Loans", "AVG_Dederal_Student_Loans_Amount", "Total_Cases", "Percent_of_State_Cases", "Cases_per_100000" and "Total_Deaths" are numerical variable. Let's figure out what makes them non-numeric to convert

In [5]:
df[df["Total_Federal_Student_Loans"]=="(*)"]

,Control,State,Plan,2018 Fall Enrollment,Known_Cases_in_County_per_100k_Residents,lat,lng,institution,Address,Campus_Size,Housing,Total_All_Undergrad_Grant/Scholarship,AVG_All_Undergrad_Grant/Scholarship,Total_Federal_Student_Loans,AVG_Dederal_Student_Loans_Amount,undergraduate_population,graduate_population,total_number_faculty,college_year,zipcode,city,county,Tuition_out_state_undergraduate,Tuition_in_state_undergraduate,Tuition_In_state_graduate,Tuition_Out_State_Graduate,Tuition_fee_graduate_In_State,Tuition_fee_graduate_Out_State,Average_Tuition,Average_Expense,AVG_Tuition-Grad-2019-2020,AVG_Fee-Grad-2019-2020,per_dem,per_gop,pi_2014,pi_2015,pi_2016,pi_2017,pi_2018,gdp_2014,gdp_2015,gdp_2016,gdp_2017,gdp_2018,state,cases_mar,cases_apr,cases_may,cases_june,cases_july,death_mar,death_apr,death_may,death_june,death_july,Total_Cases,Perecent_of_State_Cases,Cases_per_100000,Total_Deaths
1667,Private,AL,TBD,78.0,1146,34.856138,-87.662975,Heritage Christian University,"3625 Helton Dr, Florence, Alabama 35630-9977",City: Small,Yes,401144.0,8187.0,(*),(*),59,27,4,4,35630,Florence,Lauderdale,0.0,0.0,0,0,0,0,10947.0,19844.25,4482.0,630.0,0.254477,0.714580,34109.0,35009.0,35202.0,35573.0,37151.0,2479646.0,2445200.0,2402567.0,2395624.0,2467086.0,AL,12,38,161,412,996,1,2,2,5,10,1135,1.2%,"1,224",<20
1813,Public,CA,Primarily online,2404.0,341,38.926983,-119.972798,Lake Tahoe Community College,"One College Drive, South Lake Tahoe, Californi...",Rural: Fringe,Yes,2562244.0,1921.0,(*),(*),2577,0,179,2,96150,South Lake Tahoe,El Dorado,7600.5,1224.0,0,0,0,0,0.0,19998.00,NaN,NaN,0.394977,0.533835,57220.0,60087.0,62864.0,64519.0,67464.0,6417640.0,6601749.0,7043191.0,7419868.0,7653908.0,CA,15,44,90,184,613,0,0,0,0,1,716,0.1%,371,<20


Looks like the numeric values in the column were recorded as string and there are rows with (*) for schools that does not have federal student loans causing the column to be coded as Object. Let's fix it.

In [6]:
df["Total_Federal_Student_Loans"]=df["Total_Federal_Student_Loans"].str.strip()
df["Total_Federal_Student_Loans"].replace("(*)",0,inplace = True)
df["Total_Federal_Student_Loans"]= df["Total_Federal_Student_Loans"].astype("float")

"AVG_Dederal_Student_Loans_Amount" has the same issue. Let's fix it

In [7]:
df.rename(columns = {"AVG_Dederal_Student_Loans_Amount":"AVG_Federal_Student_Loans_Amount"},inplace=True)
df["AVG_Federal_Student_Loans_Amount"].fillna("0",inplace = True)
df["AVG_Federal_Student_Loans_Amount"]=df["AVG_Federal_Student_Loans_Amount"].str.strip()
df["AVG_Federal_Student_Loans_Amount"].replace(["(*)","_"],0,inplace = True)
df["AVG_Federal_Student_Loans_Amount"]= df["AVG_Federal_Student_Loans_Amount"].astype("float")

The four columns related to Covid19 from the CDC "Total_Cases", "Percent_of_State_Cases", "Cases_per_100000", "Total_Deaths" contain value such as <20, not calculated. Let's fix them

In [8]:
df["Total_Cases"] = df["Total_Cases"].apply(lambda x: "19" if x == "<20" else x).astype("float")
df.rename(columns = {'Perecent_of_State_Cases':'Percent_of_State_Cases'},inplace = True)
df["Percent_of_State_Cases"] =df["Percent_of_State_Cases"].apply(lambda x: "0" if x=="Not Calculated" else x).str.replace("%","").astype("float")
df["Cases_per_100000"] =df["Cases_per_100000"].apply(lambda x: "0" if x=="Not Calculated" else x).str.replace(",","").astype("float")
df["Total_Deaths"] = df["Total_Deaths"].apply(lambda x: "19" if x == "<20" else x).str.replace(",","").astype("float")

Now we have all of the columns wrongly coded as object recoded to numeric. Let's inpsect the rest of the "Object" type columns

In [9]:
df.select_dtypes(include = "object").columns

Index(['Control', 'State', 'Plan', 'institution', 'Address', 'Campus_Size',
       'Housing', 'college_year', 'city', 'county', 'state'],
      dtype='object')

In [10]:
df["Control"].unique()

array(['Public', 'Private'], dtype=object)

In [11]:
df["Plan"].unique()

array(['TBD', 'Primarily in person', 'Hybrid', 'Primarily online',
       'Other', 'Fully online', 'Fully in person'], dtype=object)

We can see that schools' Fall reopening plan fall into 4 main categories such as: In-person(Primarily in person, Fully in person), Hybird, Online (Primarily online,"Fully online") and Others(Other, TBD). Let's remap the plan into the main cateogies so we can easily see the trend of school's decision



In [12]:
df["Plan"] = df.Plan.map({"Primarily in person":"In-person",
                               "Fully in person":"In-person",
                               "Hybrid":"Hybrid",
                               "Primarily online":"Online",
                               "Fully online":"Online",
                               "Other":"Other",
                               "TBD":"TBD"})

In [15]:
df.Plan.unique()

array(['TBD', 'In-person', 'Hybrid', 'Online', 'Other'], dtype=object)

In [16]:
df.Campus_Size.unique()

array(['Rural: Remote', 'City: Midsize', 'Town: Distant', 'Suburb: Large',
       'City: Small', 'Town: Remote', 'City: Large', 'Rural: Fringe',
       'Suburb: Midsize', 'Town: Fringe', 'Suburb: Small',
       'Rural: Distant'], dtype=object)

Looks like Campus Size column can be splitted to two variables. Let's create variable Campus_Location and Campus_Size to better represent this data

In [17]:
df["Campus_Location"]= df.Campus_Size.str.split(":").apply(lambda x: x[0])
df["Campus_Size"]= df.Campus_Size.str.split(":").apply(lambda x: x[1]).str.strip()
df["Campus_Size"].unique()

array(['Remote', 'Midsize', 'Distant', 'Large', 'Small', 'Fringe'],
      dtype=object)

In [18]:
df["Campus_Location"].unique()

array(['Rural', 'City', 'Town', 'Suburb'], dtype=object)

In [19]:
df.select_dtypes(include = "object").columns

Index(['Control', 'State', 'Plan', 'institution', 'Address', 'Campus_Size',
       'Housing', 'college_year', 'city', 'county', 'state',
       'Campus_Location'],
      dtype='object')

In [171]:
df["college_year"].unique()

array(['2', '4', '<2'], dtype=object)

In [20]:
df["college_year"].replace("<2","2",inplace = True)

### Handling Numeric Columns

We have year to year gdp, personal income data in the dataset. Let's combine the year to year into one single column for each variable by calculating the average percentage change.

In [173]:
df["pct_change_2014_2015_gdp"] = (df["gdp_2015"]-df["gdp_2014"])/df["gdp_2014"]
df["pct_change_2015_2016_gdp"] = (df["gdp_2016"]-df["gdp_2015"])/df["gdp_2015"]
df["pct_change_2016_2017_gdp"] = (df["gdp_2017"]-df["gdp_2016"])/df["gdp_2016"]
df["pct_change_2017_2018_gdp"] = (df["gdp_2018"]-df["gdp_2017"])/df["gdp_2017"]
df["pct_change_gdp_avg"] = (df["pct_change_2014_2015_gdp"]+df["pct_change_2015_2016_gdp"]+df["pct_change_2016_2017_gdp"]+df["pct_change_2017_2018_gdp"])/4

df["pct_change_2014_2015_income"] = (df["pi_2015"]-df["pi_2014"])/df["pi_2014"]
df["pct_change_2015_2016_income"] = (df["pi_2016"]-df["pi_2015"])/df["pi_2015"]
df["pct_change_2016_2017_income"] = (df["pi_2017"]-df["pi_2016"])/df["pi_2016"]
df["pct_change_2017_2018_income"] = (df["pi_2018"]-df["pi_2017"])/df["pi_2017"]
df["pct_change_income_avg"] = (df["pct_change_2014_2015_income"] +df["pct_change_2015_2016_income"]+df["pct_change_2016_2017_income"]+df["pct_change_2017_2018_income"])/4

Inspecting Missing Value

In [174]:
df.isnull().sum().sort_values(ascending= False).head()

AVG_Fee-Grad-2019-2020        1656
AVG_Tuition-Grad-2019-2020    1656
pct_change_income_avg            0
Tuition_In_state_graduate        0
zipcode                          0
dtype: int64

In [175]:
df[df["AVG_Fee-Grad-2019-2020"].isnull()].head()

,Control,State,Plan,2018 Fall Enrollment,Known_Cases_in_County_per_100k_Residents,lat,lng,institution,Address,Campus_Size,Housing,Total_All_Undergrad_Grant/Scholarship,AVG_All_Undergrad_Grant/Scholarship,Total_Federal_Student_Loans,AVG_Federal_Student_Loans_Amount,undergraduate_population,graduate_population,total_number_faculty,college_year,zipcode,city,county,Tuition_out_state_undergraduate,Tuition_in_state_undergraduate,Tuition_In_state_graduate,Tuition_Out_State_Graduate,Tuition_fee_graduate_In_State,Tuition_fee_graduate_Out_State,Average_Tuition,Average_Expense,AVG_Tuition-Grad-2019-2020,AVG_Fee-Grad-2019-2020,per_dem,per_gop,pi_2014,pi_2015,pi_2016,pi_2017,pi_2018,gdp_2014,gdp_2015,gdp_2016,gdp_2017,gdp_2018,state,cases_mar,cases_apr,cases_may,cases_june,cases_july,death_mar,death_apr,death_may,death_june,death_july,Total_Cases,Percent_of_State_Cases,Cases_per_100000,Total_Deaths,Plan_type,Campus_Location,pct_change_2014_2015_gdp,pct_change_2015_2016_gdp,pct_change_2016_2017_gdp,pct_change_2017_2018_gdp,pct_change_gdp_avg,pct_change_2014_2015_income,pct_change_2015_2016_income,pct_change_2016_2017_income,pct_change_2017_2018_income,pct_change_income_avg
0,Public,MT,TBD,150.0,134,48.483856,-108.757664,Aaniiih Nakoda College,"269 Blackfeet Avenue Agency, Harlem, Montana 5...",Remote,No,819301.0,8716.0,0.0,0.0,131,0,23,2,59526,Harlem,Blaine,0.00,0.0,0,0,0,0,2410.0,17030.0,NaN,NaN,0.452881,0.480351,29084.0,29650.0,28958.0,29329.0,31263.0,211962.0,205467.0,181000.0,164944.0,197201.0,MT,0,0,0,0,7,0,0,0,0,0,19.0,0.0,0.0,0.0,Not In-Class,Rural,-0.030642,-0.119080,-0.088707,0.195563,-0.010717,0.019461,-0.023339,0.012812,0.065942,0.018719
4,Public,GA,Online,4291.0,3076,31.483076,-83.530194,Abraham Baldwin Agricultural College,"2802 Moore Hwy, Tifton, Georgia 31793-2601",Distant,Yes,16222202.0,4953.0,8519194.0,5679.0,3927,0,201,4,31793,Tifton,Tift,10253.25,3506.0,0,0,0,0,0.0,15242.5,NaN,NaN,0.304158,0.678337,33648.0,36633.0,35361.0,36148.0,37805.0,1631881.0,1803154.0,1641382.0,1671954.0,1722446.0,GA,19,118,256,750,1208,0,5,17,26,34,1297.0,0.6,3191.0,35.0,Not In-Class,Town,0.104954,-0.089716,0.018626,0.030199,0.016016,0.088713,-0.034723,0.022256,0.045839,0.030521
5,Private,FL,Other,466.0,2367,26.706423,-80.121485,Academy for Nursing and Health Occupations,"5154 Okeechobee Blvd Ste 201, West Palm Beach,...",Large,No,1577912.0,6440.0,3156855.0,12189.0,552,0,26,2,33417,West Palm Beach,Palm Beach,0.00,0.0,0,0,0,0,38150.0,91262.5,NaN,NaN,0.565111,0.411971,66126.0,70213.0,70241.0,75183.0,79760.0,62416939.0,65902060.0,67751333.0,70558459.0,72270528.0,FL,462,2961,5896,14148,32694,10,186,337,510,806,35737.0,7.0,2388.0,892.0,Not In-Class,Suburb,0.055836,0.028061,0.041433,0.024265,0.037399,0.061806,0.000399,0.070358,0.060878,0.048360
6,Public,FL,Online,29772.0,2367,26.374988,-80.101063,Florida Atlantic University,"777 Glades Rd, Administration Bldg., Room 339,...",Small,Yes,131759569.0,9031.0,50902616.0,6453.0,24740,5321,1484,4,33431,Boca Raton,Palm Beach,17288.00,4843.0,5467,16695,1226,1226,0.0,23924.5,NaN,NaN,0.565111,0.411971,66126.0,70213.0,70241.0,75183.0,79760.0,62416939.0,65902060.0,67751333.0,70558459.0,72270528.0,FL,462,2961,5896,14148,32694,10,186,337,510,806,35737.0,7.0,2388.0,892.0,Not In-Class,City,0.055836,0.028061,0.041433,0.024265,0.037399,0.061806,0.000399,0.070358,0.060878,0.048360
9,Public,FL,Online,31816.0,2367,26.840894,-80.079251,Palm Beach State College,"4200 Congress Ave, Lake Worth, Florida 33461-4796",Large,No,62084169.0,4463.0,10901708.0,4405.0,31289,0,1252,4,33461,Lake Worth,Palm Beach,8732.00,2444.0,0,0,0,0,0.0,18464.0,NaN,NaN,0.565111,0.411971,66126.0,70213.0,70241.0,75183.0,79760.0,62416939.0,65902060.0,67751333.0,70558459.0,72270528.0,FL,462,2961,5896,14148,32694,10,186,337,510,806,35737.0,7.0,2388.0,892.0,Not In-Class,Suburb,0.055836,0.028061,0.041433,0.024265,0.037399,0.061806,0.000399,0.070358,0.060878,0.048360


In [176]:
df.isnull().sum().sort_values(ascending= False).head()

AVG_Fee-Grad-2019-2020        1656
AVG_Tuition-Grad-2019-2020    1656
pct_change_income_avg            0
Tuition_In_state_graduate        0
zipcode                          0
dtype: int64

Since our dataset contains many schools that do not offer graduate program, filling missing value or delete these schools does not make sense. Therefore, we wont be using Graduate Program Expenses as variable for our model

In [177]:
df.drop(["AVG_Fee-Grad-2019-2020","AVG_Tuition-Grad-2019-2020"],axis = 1,inplace = True)

In [178]:
df.isnull().sum().sort_values(ascending= False).head()

pct_change_income_avg        0
Tuition_In_state_graduate    0
college_year                 0
zipcode                      0
city                         0
dtype: int64

In [179]:
df.drop_duplicates(subset = ("institution"),keep=False,inplace = True)
df[df.duplicated(keep = False)]

,Control,State,Plan,2018 Fall Enrollment,Known_Cases_in_County_per_100k_Residents,lat,lng,institution,Address,Campus_Size,Housing,Total_All_Undergrad_Grant/Scholarship,AVG_All_Undergrad_Grant/Scholarship,Total_Federal_Student_Loans,AVG_Federal_Student_Loans_Amount,undergraduate_population,graduate_population,total_number_faculty,college_year,zipcode,city,county,Tuition_out_state_undergraduate,Tuition_in_state_undergraduate,Tuition_In_state_graduate,Tuition_Out_State_Graduate,Tuition_fee_graduate_In_State,Tuition_fee_graduate_Out_State,Average_Tuition,Average_Expense,per_dem,per_gop,pi_2014,pi_2015,pi_2016,pi_2017,pi_2018,gdp_2014,gdp_2015,gdp_2016,gdp_2017,gdp_2018,state,cases_mar,cases_apr,cases_may,cases_june,cases_july,death_mar,death_apr,death_may,death_june,death_july,Total_Cases,Percent_of_State_Cases,Cases_per_100000,Total_Deaths,Plan_type,Campus_Location,pct_change_2014_2015_gdp,pct_change_2015_2016_gdp,pct_change_2016_2017_gdp,pct_change_2017_2018_gdp,pct_change_gdp_avg,pct_change_2014_2015_income,pct_change_2015_2016_income,pct_change_2016_2017_income,pct_change_2017_2018_income,pct_change_income_avg


In [180]:
public_drop = df[(df["Control"]=="Public") & (df["Tuition_out_state_undergraduate"]==0)][["institution","Control","Tuition_out_state_undergraduate","Tuition_in_state_undergraduate","Average_Tuition","Average_Expense"]].index
df.drop(index = public_drop,inplace = True)
df[(df["Control"]=="Public") & (df["Tuition_out_state_undergraduate"]==0)][["institution","Control","Tuition_out_state_undergraduate","Tuition_in_state_undergraduate","Average_Tuition","Average_Expense"]]

,institution,Control,Tuition_out_state_undergraduate,Tuition_in_state_undergraduate,Average_Tuition,Average_Expense


In [181]:
private_drop = df[(df["Control"]=="Private") & (df["Average_Tuition"]==0)][["institution","Control","Tuition_out_state_undergraduate","Tuition_in_state_undergraduate","Average_Tuition","Average_Expense"]].index
df.drop(index = private_drop,inplace = True)
df[(df["Control"]=="Private") & (df["Average_Tuition"]==0)][["institution","Control","Tuition_out_state_undergraduate","Tuition_in_state_undergraduate","Average_Tuition","Average_Expense"]]

,institution,Control,Tuition_out_state_undergraduate,Tuition_in_state_undergraduate,Average_Tuition,Average_Expense


In [182]:
df.reset_index(inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1947 entries, 0 to 1946
Data columns (total 70 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   index                                     1947 non-null   int64  
 1   Control                                   1947 non-null   object 
 2   State                                     1947 non-null   object 
 3   Plan                                      1947 non-null   object 
 4   2018 Fall Enrollment                      1947 non-null   float64
 5   Known_Cases_in_County_per_100k_Residents  1947 non-null   int64  
 6   lat                                       1947 non-null   float64
 7   lng                                       1947 non-null   float64
 8   institution                               1947 non-null   object 
 9   Address                                   1947 non-null   object 
 10  Campus_Size                         

In [183]:
df.to_csv("final_0902_edited.csv")

In [153]:
df[(df["Control"]=="Public") & (df["Tuition_out_state_undergraduate"]==0)][["institution","Control","Tuition_out_state_undergraduate","Tuition_in_state_undergraduate","Average_Tuition","Average_Expense"]]

,institution,Control,Tuition_out_state_undergraduate,Tuition_in_state_undergraduate,Average_Tuition,Average_Expense
0,Aaniiih Nakoda College,Public,0.0,0.0,2410.00000,17030.00
12,New York Institute of Technology,Public,0.0,0.0,36482.50000,57596.25
65,Alcorn State University,Public,0.0,0.0,6952.00000,23762.00
67,Alexandria Technical and Community College,Public,0.0,0.0,5437.25000,18937.25
82,Allen County Community College,Public,0.0,0.0,3330.50000,13491.00
134,Hacienda La Puente Adult Education,Public,0.0,0.0,3324.00000,20575.00
261,Anoka Technical College,Public,0.0,0.0,5609.00000,21252.50
262,Anoka-Ramsey Community College,Public,0.0,0.0,5079.50000,20723.25
305,University of Arizona,Public,0.0,0.0,19338.75000,39583.75
311,University of Arkansas -Little Rock,Public,0.0,0.0,0.00000,23679.25


In [ ]:
df["Average_Tuition","Average_Expense"]